In [1]:
import time

from IPython.display import Markdown, display  # Import
from openai import OpenAI
import warnings
import tiktoken
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
from dotenv import load_dotenv
import asyncio
import pandas as pd
from io import StringIO
from src.business_requirements import *
from src.prompts import *
from src.simplified_business_requirements import *
warnings.filterwarnings("ignore")
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")


## Query Question

In [2]:
query = "What are the business requirements for Regulation K?"
response = verbatim_business_requirements(query, top_k=25)
display(Markdown(response))

"Business Requirement"  
"No foreign bank may establish a branch or an agency, or acquire ownership or control of a commercial lending company, without the prior approval of the Board."  
"A foreign bank must receive a license from the OCC to open and operate its initial Federal branch or agency in the United States."  
"Each branch or agency of a foreign bank, other than a Federal branch or agency, shall be subject to section 335 of this title and the provision requiring the reports of condition contained in section 324 of this title, as if the branch or agency were a State member bank."  
"A banking institution shall establish an allocated transfer risk reserve (ATRR) for specified international assets when required by the Board in accordance with this section."  
"A national bank shall notify the OCC when it files an application, notice, or report with the FRB to establish or open a foreign branch, acquire or divest an interest in, or close, an Edge corporation, Agreement corporation, foreign bank, or other foreign organization."  
"A national bank shall file any notice or submission required under this subpart with the appropriate supervisory office of the OCC."  
"With the Board's specific approval, an investor may engage in other activities that the Board determines are usual in connection with the transaction of the business of banking or other financial operations abroad and are consistent with the FRA or the BHC Act."  
"Any person may submit a request to the Board for an advisory opinion regarding the scope of activities permissible under Regulation K."  
"The Board shall provide an advisory opinion within 45 days after the record on the request for an advisory opinion has been determined to be complete."  
"No banking institution shall charge, in connection with the restructuring of an international loan, any fee exceeding the administrative cost of the restructuring unless it amortizes the fee over the effective life of the loan."  
"Banking institutions shall account for fees on international loans in accordance with generally accepted accounting principles."  
"The Board may disclose information obtained in the course of exercising its supervisory or examination authority to a foreign bank regulatory or supervisory authority if the Board determines that disclosure is appropriate for supervisory or regulatory purposes and will not prejudice the interests of the United States."  
"Before making any disclosure of information pursuant to the Board's supervisory authority, the Board shall obtain, to the extent necessary, the agreement of the foreign bank regulatory or supervisory authority to maintain the confidentiality of such information."

## Create Excel Sheet

In [5]:
def clean_dataframe(df, column):
    """
    Cleans the DataFrame by:
      - Dropping rows in which the specified column does not contain any alphabetical characters (A-Z, a-z).
      - Removing the substring "| " from the specified column.

    Parameters:
      df (pd.DataFrame): The input DataFrame.
      column (str): The name of the column to clean.

    Returns:
      pd.DataFrame: The cleaned DataFrame.
    """
    # Ensure the column values are treated as strings.
    df[column] = df[column].astype(str)

    # Keep only rows where the column contains at least one alphabetical character.
    df = df[df[column].str.contains(r'[A-Za-z]', na=False)]

    # Remove the exact substring "| " from the column.
    df[column] = df[column].str.replace("| ", "", regex=False)

    # Remove the exact substring "|" from the column.
    df[column] = df[column].str.replace("|", "", regex=False)

    # Remove the exact substring "^^" from the column.
    df[column] = df[column].str.replace("^^", "", regex=False)

    # Remove the exact substring "" " from the column.
    df[column] = df[column].str.replace('''"''', "", regex=False)
    return df

df = convert_str_to_df(response)

# Clean Table
df = df.dropna(axis=1, how='all')
df = clean_dataframe(df, df.columns[0])
df["Business_Requirements"] = df[df.columns[0]]
df = df.drop(df.columns[0], axis=1)
df


,Business_Requirements
0,No foreign bank may establish a branch or an a...
1,A foreign bank must receive a license from the...
2,"Each branch or agency of a foreign bank, other..."
3,A banking institution shall establish an alloc...
4,A national bank shall notify the OCC when it f...
5,A national bank shall file any notice or submi...
6,"With the Board's specific approval, an investo..."
7,Any person may submit a request to the Board f...
8,The Board shall provide an advisory opinion wi...
9,"No banking institution shall charge, in connec..."


## Create Simplified Business Requirements

In [8]:
async def simplified_business_requirement(query: str, df: pd.DataFrame):
    # Create asynchronous tasks for each row's business requirement
    tasks = [
        simplified_buss_req(original_question=query, business_requirement=row)
        for row in df["Business_Requirements"]
    ]

    # Execute all tasks concurrently and collect results
    results = await asyncio.gather(*tasks)

    # Store the simplified business requirements back into the DataFrame
    df["Simplified_Business_Requirements"] = results
    return df

# Run the asynchronous function with your query and DataFrame
await simplified_business_requirement(query=query, df=df)
df

,Business_Requirements,Simplified_Business_Requirements
0,No foreign bank may establish a branch or an a...,The business requirement for Regulation K stat...
1,A foreign bank must receive a license from the...,A foreign bank must receive a license from the...
2,"Each branch or agency of a foreign bank, other...",The business requirements for Regulation K sta...
3,A banking institution shall establish an alloc...,The business requirement for Regulation K stat...
4,A national bank shall notify the OCC when it f...,A national bank must notify the OCC when it fi...
5,A national bank shall file any notice or submi...,A national bank shall file any notice or submi...
6,"With the Board's specific approval, an investo...",The business requirement for Regulation K stat...
7,Any person may submit a request to the Board f...,The business requirement for Regulation K stat...
8,The Board shall provide an advisory opinion wi...,The business requirement for Regulation K stat...
9,"No banking institution shall charge, in connec...",The business requirement for Regulation K stat...
